In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [2]:
Main_df = pd.read_csv('weatherAUS.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'weatherAUS.csv'

In [ ]:
Main_df

In [ ]:
Main_df.info()

In [ ]:
Main_df.dropna(subset=['RainToday','RainTomorrow'], inplace=True)

In [ ]:
Main_df.info()

In [ ]:
import plotly.express as px
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

sns.set_style('darkgrid')
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['figure.figsize'] = (10, 6)
matplotlib.rcParams['figure.facecolor'] = '#00000000'

In [ ]:
px.histogram(Main_df, x='Location', title='Location vs. Rainy Days', color='RainToday')

In [ ]:
from sklearn.model_selection import train_test_split

train_val_df, test_df = train_test_split(Main_df, test_size=0.2, random_state=42)
train_df, val_df = train_test_split(train_val_df, test_size=0.25, random_state=42)

print('train_df.shape :', train_df.shape)
print('val_df.shape :', val_df.shape)
print('test_df.shape :', test_df.shape)

In [ ]:
plt.title('No. of Rows per Year')
sns.countplot(x=pd.to_datetime(Main_df.Date).dt.year);

In [ ]:
year = pd.to_datetime(Main_df['Date']).dt.year

df_train = Main_df[year < 2015]
df_val = Main_df[year == 2015]
df_test = Main_df[year > 2015]

In [ ]:
input_cols = list(df_train.columns)[1:-1]
target_col = list(df_train.columns)[-1]
print(input_cols)
print(target_col)

In [ ]:
train_input = df_train[input_cols].copy()
train_target = df_train[target_col].copy()

In [ ]:
val_input = df_val[input_cols].copy()
val_target = df_val[target_col].copy()

In [ ]:
test_input = df_test[input_cols].copy()
test_target = df_test[target_col].copy()

In [ ]:
numeric_cols = train_input.dtypes[train_input.dtypes != 'object'].index
categorical_cols = train_input.dtypes[train_input.dtypes == 'object'].index

In [ ]:
categorical_cols

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
imputer = SimpleImputer(strategy='mean')

In [ ]:
imputer.fit(Main_df[numeric_cols])

In [ ]:
train_input[numeric_cols] = imputer.transform(train_input[numeric_cols])
val_input[numeric_cols] = imputer.transform(val_input[numeric_cols])
test_input[numeric_cols] = imputer.transform(test_input[numeric_cols])

In [ ]:
train_input[numeric_cols].isna().sum()

In [ ]:
Main_df[numeric_cols].describe()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(Main_df[numeric_cols])

In [ ]:
print('Minimum:')
list(scaler.data_min_)

In [ ]:
print('Maximum:')
list(scaler.data_max_)

In [ ]:
train_input[numeric_cols] = scaler.transform(train_input[numeric_cols])
val_input[numeric_cols] = scaler.transform(val_input[numeric_cols])
test_input[numeric_cols] = scaler.transform(test_input[numeric_cols])

In [ ]:
train_input[numeric_cols].describe()

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
Main_df[categorical_cols].nunique()

In [ ]:
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

In [ ]:
encoder.fit(Main_df[categorical_cols])

In [ ]:
encoded_cols = list(encoder.get_feature_names_out(categorical_cols))
print(encoded_cols)

In [ ]:
train_input[encoded_cols] = encoder.transform(train_input[categorical_cols])
val_input[encoded_cols] = encoder.transform(val_input[categorical_cols])
test_input[encoded_cols] = encoder.transform(test_input[categorical_cols])

In [ ]:
pd.set_option('display.max_columns', None)
train_input

In [ ]:
print('train_input:', train_input.shape)
print('train_target:', train_target.shape)
print('val_input:', val_input.shape)
print('val_target:', val_target.shape)
print('test_input:', test_input.shape)
print('test_target:', test_target.shape)

In [ ]:
train_input.to_parquet('train_input.parquet')
val_input.to_parquet('val_input.parquet')
test_input.to_parquet('test_input.parquet')

In [ ]:
%time
pd.DataFrame(train_target).to_parquet('train_target.parquet')
pd.DataFrame(val_target).to_parquet('val_target.parquet')
pd.DataFrame(test_target).to_parquet('test_target.parquet')

In [ ]:
%%time

train_input = pd.read_parquet('train_input.parquet')
val_input = pd.read_parquet('val_input.parquet')
test_input = pd.read_parquet('test_input.parquet')

train_target = pd.read_parquet('train_target.parquet')[target_col]
val_target = pd.read_parquet('val_target.parquet')[target_col]
test_target = pd.read_parquet('test_target.parquet')[target_col]

In [ ]:
print('train_input:', train_input.shape)
print('train_target:', train_target.shape)
print('val_input:', val_input.shape)
print('val_target:', val_target.shape)
print('test_input:', test_input.shape)
print('test_target:', test_target.shape)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
model = LogisticRegression(solver='liblinear')

In [ ]:
train_input = train_input.drop(categorical_cols, axis=1)
val_input = val_input.drop(categorical_cols, axis=1)
test_input = test_input.drop(categorical_cols, axis=1)


train_input = pd.concat([train_input[numeric_cols], train_input[encoded_cols]], axis=1)
val_input = pd.concat([val_input[numeric_cols], val_input[encoded_cols]], axis=1)
test_input = pd.concat([test_input[numeric_cols], test_input[encoded_cols]], axis=1)

model.fit(train_input, train_target)

In [ ]:
print(list(numeric_cols) + encoded_cols)

In [ ]:
print(model.coef_.tolist())

In [ ]:
plt.figure(figsize = (10,10))

# Create a DataFrame of feature importances (absolute coefficients)
feature_importance = pd.DataFrame({
    'Feature': list(numeric_cols) + encoded_cols,
    'Coefficient': model.coef_[0]
})

# Sort by absolute coefficient values for visualization
feature_importance['Abs_Coefficient'] = abs(feature_importance['Coefficient'])
feature_importance = feature_importance.sort_values('Abs_Coefficient', ascending=False)

# Select the top 10 features
top_10_features = feature_importance.head(15)

# Create the barplot using the new DataFrame
sns.barplot(data=top_10_features, x='Coefficient', y='Feature');

In [ ]:
X_train = train_input[list(numeric_cols) + encoded_cols]
X_val = val_input[list(numeric_cols) + encoded_cols]
X_test = test_input[list(numeric_cols) + encoded_cols]

In [ ]:
train_preds = model.predict(X_train)

In [ ]:
train_preds

In [ ]:
train_target

In [ ]:
train_probs = model.predict_proba(X_train)
train_probs

In [ ]:
model.classes_

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(train_target, train_preds)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(train_target, train_preds, normalize='true')

In [ ]:
def predict_and_plot(inputs, targets, name=''):
    preds = model.predict(inputs)

    accuracy = accuracy_score(targets, preds)
    print("Accuracy: {:.2f}%".format(accuracy * 100))

    cf = confusion_matrix(targets, preds, normalize='true')
    plt.figure()
    sns.heatmap(cf, annot=True)
    plt.xlabel('Prediction')
    plt.ylabel('Target')
    plt.title('{} Confusion Matrix'.format(name));

    return preds

In [ ]:
train_preds = predict_and_plot(X_train, train_target, 'Training')

In [ ]:
import joblib

In [ ]:
aussie_rain = {
    'model': model,
    'imputer': imputer,
    'scaler': scaler,
    'encoder': encoder,
    'input_cols': input_cols,
    'target_col': target_col,
    'numeric_cols': numeric_cols,
    'categorical_cols': categorical_cols,
    'encoded_cols': encoded_cols
}

In [ ]:
joblib.dump(aussie_rain, 'aussie_rain.joblib')

In [ ]:
aussie_rain2 = joblib.load('aussie_rain.joblib')

In [ ]:
test_preds2 = aussie_rain2['model'].predict(X_test)
accuracy_score(test_targets, test_preds2)